# Baseline Notebook

# Dependencies

In [1]:
import numpy as np
import os

#Import torch stuff.
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

#Append the path outside so we can load bom1.
import sys
sys.path.append('..')

import bom1.wakeword as wf
import bom1.bom1 as bom1

import matplotlib.pyplot as plt
from tqdm import tqdm

Set the training path.

In [2]:
train_path  = '/work3/s164419/01005WakeWordData/1_to_5_1s_train/train/'
train_files = [os.path.join(train_path, x) for x in os.listdir(train_path) if x.endswith('.wav')]

Set the validation path.

In [3]:
val_path  = '/work3/s164419/01005WakeWordData/every50_1s_val_test/val'
val_files = [os.path.join(val_path, x) for x in os.listdir(val_path) if x.endswith('.wav')]

In [4]:
#Set the functions for loading the data.
_, _, spectrogram = wf.load_data(train_files[0], f = T.Spectrogram(hop_length=40), normalize=True, transforms=[wf.TransformMono(), wf.Padder(22050)])

#Flatten the spectrogram
spectrogram = spectrogram.flatten().numpy()

X_train = np.zeros(shape=(len(train_files), len(spectrogram)))
y_train = np.zeros(shape=len(train_files))

In [8]:
for i, path in tqdm(enumerate(train_files), desc='Loading training', total=len(train_files)):
    _, _, spectrogram = wf.load_data(path, f = T.Spectrogram(hop_length=40), normalize=True, transforms=[wf.TransformMono(), wf.Padder(22050)])
    
    #Flatten and convert to numpy
    spectrogram = spectrogram.flatten().numpy().astype('float16')

    X_train[i,:] = spectrogram
    
    #Extract the target
    _, _, _, target = wf.info_from_path(path)

    y_train[i] = target

Loading training:  86%|████████▋ | 21976/25454 [04:13<00:39, 88.31it/s]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
pipe = Pipeline([('LogisticRegression', LogisticRegression(n_jobs=-1))])
pipe.fit(X_train, y_train)